# <center><span style='background:yellow'> Participez à la conception d'une voiture autonome</span></center>
# <center><span style='background:yellow'>Comparaison des modèles</span></center>


### Table of Contents <a class="anchor" id="sommaire"></a>

* [Import et chargement des données](#0)
* [Définition des classes et fonctions communes](#00)
* [Chargements des modèles](#1)
* [Evaluation des modèles](#2)

## Import et chargement des données <a class="anchor" id="0"></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Jan 27 10:18:57 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    57W / 400W |   7650MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import time
import pickle
import cv2
from google.colab.patches import cv2_imshow
import tensorflow as tf
import keras
import albumentations as A
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
print("Keras version : ", keras.__version__)
print("Tensorflow version : ", tf.__version__)

Keras version :  2.9.0
Tensorflow version :  2.9.2


In [ ]:
data_path = "/content/drive/MyDrive/data/"
checkpoint_filepath = '/content/drive/MyDrive/checkpoint_unet/'

In [ ]:
# Chargement de données
test_path = pd.read_pickle(data_path + 'test_path.pkl')

x_test = np.load(data_path + 'x_test.npy')/255.
y_test = np.load(data_path + 'y_test.npy')

with open(checkpoint_filepath + 'unet_times.pkl', 'rb') as f:
    times = pickle.load(f)

with open(checkpoint_filepath + 'vgg_times.pkl', 'rb') as f:
    vgg_times = pickle.load(f)

In [ ]:
print("Dimension de xtest :", x_test.shape)
print("Dimension de ytest :", y_test.shape)

Dimension de xtest : (150, 128, 256, 3)
Dimension de ytest : (150, 128, 256, 8)


## Définition des classes et des fonctions

### Définition des Loss functions

In [ ]:
from tensorflow.python.keras import backend as K

def weighted_cross_entropy(beta):
  def loss(y_true, y_pred):
    weight_a = beta * tf.cast(y_true, tf.float32)
    weight_b = 1 - tf.cast(y_true, tf.float32)
    
    o = (tf.math.log1p(tf.exp(-tf.abs(y_pred))) + tf.nn.relu(-y_pred)) * (weight_a + weight_b) + y_pred * weight_b 
    return tf.reduce_mean(o)

  return loss

def balanced_cross_entropy(beta):
  def loss(y_true, y_pred):
    weight_a = beta * tf.cast(y_true, tf.float32)
    weight_b = (1 - beta) * tf.cast(1 - y_true, tf.float32)
    
    o = (tf.math.log1p(tf.exp(-tf.abs(y_pred))) + tf.nn.relu(-y_pred)) * (weight_a + weight_b) + y_pred * weight_b
    return tf.reduce_mean(o)

  return loss


def dice_loss(y_true, y_pred):
  y_true = tf.cast(y_true, tf.float32)
  y_pred = tf.math.sigmoid(y_pred)
  numerator = 2 * tf.reduce_sum(y_true * y_pred)
  denominator = tf.reduce_sum(y_true + y_pred)

  return 1 - numerator / denominator

def total_loss(y_true, y_pred):
  y_true = tf.cast(y_true, tf.float32)
  o = tf.nn.sigmoid_cross_entropy_with_logits(y_true, y_pred) + dice_loss(y_true, y_pred)
  return tf.reduce_mean(o)

### Définition des métriques

In [ ]:
def dice_coeff(y_true, y_pred):
    score = 1 - dice_loss(y_true, y_pred)
    return score

def IoU(y_true, y_pred):
  y_true = tf.cast(y_true, tf.float32)
  y_pred = tf.math.sigmoid(y_pred)
  intersection = tf.reduce_sum(y_true * y_pred)
  denominator = tf.reduce_sum(y_true + y_pred) - intersection

  return intersection / denominator

### Paramétres des modèles

In [ ]:
models_path = '/content/drive/MyDrive/checkpoint_unet/'

## Chargement des modèles

In [ ]:
times

{'UNET_CE': 7.411513129870097,
 'UNET_WCE': 6.615199021498362,
 'UNET_DL': 2.897370978196462,
 'UNET_TL': 7.735464672247569,
 'AUG_UNET_CE': 26.66868639389674,
 'AUG_UNET_WCE': 26.606162695089974,
 'AUG_UNET_DL': 32.84838535388311,
 'AUG_UNET_TL': 69.73193047046661}

In [ ]:
times.update(vgg_times)

In [ ]:
models_name = [
         'UNET_CE',
         'UNET_WCE',
         'UNET_DL',
         'UNET_TL',
         'VGG_UNET_CE',
         'VGG_UNET_WCE',
         'VGG_UNET_DL',
         'VGG_UNET_TL',
         ]
dic = {}
dic_aug = {}
training_times = {}
training_times_aug = {}

custom_objects = {"dice_coeff" : dice_coeff, 
                  "IoU" : IoU, 
                  "dice_loss" : dice_loss, 
                  'total_loss' : total_loss,
                  'loss' : weighted_cross_entropy(2)}

for name in models_name:
  print(name)
  model = tf.keras.models.load_model(models_path + name, 
                                     custom_objects = custom_objects)
  eval = model.evaluate(x_test, y_test)

  dic[name] = eval

  print('AUG_' + name)
  model_aug = tf.keras.models.load_model(models_path + 'AUG_' + name, 
                                     custom_objects = custom_objects)
  eval_aug = model_aug.evaluate(x_test, y_test)
  dic_aug[name] = eval_aug

  training_times[name] = times[name]
  training_times_aug[name] = times["AUG_" + name]

UNET_CE
5/5 [==============================] - 1s 49ms/step - loss: 0.0744 - accuracy: 0.8916 - dice_coeff: 0.8432 - IoU: 0.7290
AUG_UNET_CE
5/5 [==============================] - 1s 49ms/step - loss: 0.0842 - accuracy: 0.8807 - dice_coeff: 0.8359 - IoU: 0.7184
UNET_WCE
5/5 [==============================] - 1s 47ms/step - loss: 0.1179 - accuracy: 0.8808 - dice_coeff: 0.8123 - IoU: 0.6841
AUG_UNET_WCE
5/5 [==============================] - 1s 49ms/step - loss: 0.1208 - accuracy: 0.8837 - dice_coeff: 0.8204 - IoU: 0.6956
UNET_DL
5/5 [==============================] - 1s 48ms/step - loss: 0.6263 - accuracy: 0.3749 - dice_coeff: 0.3737 - IoU: 0.2298
AUG_UNET_DL
5/5 [==============================] - 1s 48ms/step - loss: 0.6157 - accuracy: 0.3701 - dice_coeff: 0.3836 - IoU: 0.2374
UNET_TL
5/5 [==============================] - 1s 49ms/step - loss: 3.1744 - accuracy: 0.7040 - dice_coeff: 0.6418 - IoU: 0.4726
AUG_UNET_TL
5/5 [==============================] - 1s 48ms/step - loss: 1.0699 - ac

In [ ]:
training_time = pd.Series(training_times, name = "training_time")
training_time_aug = pd.Series(training_times_aug, name = "training_time_aug")
times = pd.concat([training_time, training_time_aug], axis=1)
times.head()

,training_time,training_time_aug
UNET_CE,7.412,26.669
UNET_WCE,6.615,26.606
UNET_DL,2.897,32.848
UNET_TL,7.735,69.732
VGG_UNET_CE,2.445,22.058


In [ ]:
col = ["Loss", "Accuracy", "Dice_coeff", "IoU", "Loss_aug", "Accuracy_aug", "Dice_coeff_aug", "IoU_aug"]

comp = pd.concat([pd.DataFrame.from_dict(dic, orient='index', columns=col[:4]), 
                  pd.DataFrame.from_dict(dic_aug, orient='index', columns=col[4:]), 
                  times], axis=1)

In [ ]:
comp = comp[["IoU", "IoU_aug", "Dice_coeff", "Dice_coeff_aug", "Accuracy", "Accuracy_aug", "Loss", 'Loss_aug', 'training_time', 'training_time_aug']]
comp = comp.sort_values(by = "IoU")

In [ ]:
pd.options.display.float_format = '{:,.3f}'.format
comp

,IoU,IoU_aug,Dice_coeff,Dice_coeff_aug,Accuracy,Accuracy_aug,Loss,Loss_aug,training_time,training_time_aug
UNET_DL,0.230,0.237,0.374,0.384,0.375,0.370,0.626,0.616,2.897,32.848
VGG_UNET_TL,0.463,0.671,0.633,0.803,0.669,0.838,0.619,0.409,2.160,85.480
UNET_TL,0.473,0.513,0.642,0.678,0.704,0.757,3.174,1.070,7.735,69.732
UNET_WCE,0.684,0.696,0.812,0.820,0.881,0.884,0.118,0.121,6.615,26.606
VGG_UNET_CE,0.694,0.746,0.819,0.854,0.868,0.891,0.088,0.074,2.445,22.058
VGG_UNET_WCE,0.702,0.744,0.825,0.853,0.877,0.893,0.123,0.107,3.107,32.314
UNET_CE,0.729,0.718,0.843,0.836,0.892,0.881,0.074,0.084,7.412,26.669
VGG_UNET_DL,0.817,0.820,0.899,0.901,0.890,0.891,0.102,0.100,5.877,46.074


In [ ]:
path = '/content/drive/MyDrive/data/'
comp.to_csv(path + 'models_comparaison.csv', index=False, header=True, decimal=',', sep=';', float_format='{:.3f}'.format)